# Occlusion Segmentation Branch
Unet을 사용하여 마스크(가려진부분) segmentation 진행
<br/>
<br/>Implementation of Large Kernel Matters Paper (face++)
<br/>Author : Xiangtai (lxtpku@pku.edu.cn)

In [3]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import OrderedDict

import numpy as np

# TODO
# BUG
# FIXME
# HACK

In [4]:
__all__ = ['unet',]

class _GlobalConvModule(nn.Module):
    def __init__(self, in_dim, out_dim, kernel_size):
        '''
        kerenl_size : odd number인게 좋음
        '''
        super(_GlobalConvModule,self).__init__()
        pad0 = (kernel_size[0] - 1) // 2
        pad1 = (kernel_size[1] - 1) // 2
        
        # NOTE kernel size had better be odd number so as to avoid alignment error
        super(_GlobalConvModule, self).__init__()
        self.conv_l1 = nn.Conv2d(in_dim, out_dim, kernel_size=(kernel_size[0],1),padding=(pad0,0))
        self.conv_l2 = nn.Conv2d(out_dim, out_dim, kernel_size=(1, kernel_size[1]), padding=(0,pad1))
        self.conv_r1 = nn.Conv2d(in_dim, out_dim, kernel_size=(1,kernel_size[1]),padding=(0,pad1))
        self.conv_r2 = nn.Conv2d(out_dim, out_dim, kernel_size=(kernel_size[0],1), padding=(pad0,0))
    
    def forward(self, x):
        x_l = self.conv_l1(x)
        x_l = self.conv_l2(x_l)

        x_r = self.conv_r1(x)
        x_r = self.conv_r2(x_r)

        x = x_l + x_r
        return x
       

### Conv Blocks

**What is Dilation is Conv?**
<div>
<img src="../../img/DilatedConv.png" width="200"/>
</div>

In [5]:
def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    '''
    3x3 convolution with padding
    '''
    return nn.Conv2d(in_planes,
                     out_planes,
                     kernel_size=3,
                     stride=stride,
                     padding=dilation,
                     groups=groups,
                     bias=False,
                     dilation=dilation)

def conv1x1(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes,out_planes,kernel_size=1,stride=stride,bias=False)

### IBasicBlock

identity -> bn -> conv -> bn -> PReLU -> conv -> bn3 -> out -> out + identity

In [10]:
class IBasicBlock(nn.Module):
    '''identity -> bn -> conv -> bn -> PReLU -> conv -> bn3 -> out -> out + identity'''
    expansion = 1
    def __init__(self,
                 inplanes,
                 planes,
                 stride=1,
                 downsample=None,
                 groups=1,
                 base_width=64,
                 dilation=1):
        super(IBasicBlock, self).__init__()
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        
        self.bn1 = nn.BatchNorm2d(inplanes,eps=1e-05)
        self.conv1 = conv3x3(inplanes,planes)
        self.bn2 = nn.BatchNorm2d(planes, eps=1e-06)
        self.prelu = nn.PReLU(planes)
        self.conv2 = conv3x3(planes, planes, stride)
        self.bn3 = nn.BatchNorm2d(planes, eps=1e-05)
        self.downsample = downsample
        self.stride = stride
    
    def forward(self, x):
        identity = x
        out = self.bn1(x)
        out = self.conv1(out)
        out = self.bn2(out)
        out = self.prelu(out)
        out = self.conv2(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)
        out += identity
        return out

# UNET!

In [11]:
class Unet(nn.Module):
    def __init__(self,
                 block:IBasicBlock,
                 layers,
                 groups=1,
                 num_classes=2,
                 kernel_size=7,
                 dap_k=3,
                 gray=True, # important
                 input_size=128 # important
                 ):
        
        super(Unet,self).__init__()

        self.inplanes = 64
        self.dilation = 1
        self.groups = groups
        self.base_width = 64

        if gray:
            self.conv1 = nn.Conv2d(1, self.inplanes, kernel_size=3, stride=2, padding=1, bias=False)
        else:
            self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=3, stride=2, padding=1, bias=False)

        # Batch Norm
        self.bn1 = nn.BatchNorm2d(self.inplanes, eps=1e-05)
        # PReLU
        self.prelu = nn.PReLU(self.inplanes)

        #
    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        downsample = None
        previous_dilation = self.dilation

        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:

        

SyntaxError: incomplete input (3790590755.py, line 11)

In [17]:
class Test():
    def __init__(self):
        self._sayhi()
    
    def _sayhi(self):
        print("hihi")

t = Test()

hihi
